In [1]:
import gspread
import pandas as pd
import os
import numpy as np

from dotenv import load_dotenv
from google.oauth2 import service_account

# === Carregar variáveis de ambiente ===
load_dotenv()

# === Definir escopos de acesso ===
scopes = [
    "https://www.googleapis.com/auth/spreadsheets.readonly",
    "https://www.googleapis.com/auth/drive.readonly"
]

# === Caminho seguro para o arquivo JSON (lido do .env) ===
cred_path = os.getenv('GOOGLE_CREDENTIALS_PATH')

# === Carregar credenciais de forma moderna ===
creds = service_account.Credentials.from_service_account_file(
    cred_path,
    scopes=scopes
)

# === Autorizar o cliente gspread ===
client = gspread.authorize(creds)

# === Função para carregar aba da planilha ===
def carregar_aba(sheet_id, aba_nome):
    planilha = client.open_by_key(sheet_id)
    aba = planilha.worksheet(aba_nome)
    dados = aba.get_all_records()
    return pd.DataFrame(dados)

# === IDs das planilhas (fixar no .env depois se quiser) ===
id_dados_leads_ssp = "1K7nLxfgu6ktdyFwlHMPJsj5xdv410tensfPccmmPDm8"
id_pesquisa_ssp = "1pK0BP8n98DtQ8OwGeBhzcGgxQinFqZhZsQTk7_cYZIk"
id_compradores_ssp = "1A3D8JNPlZNpxkwcXQ_7CxalNEZyLo8KmJP6Sa_A-3NU"

# === Carregar DataFrames ===
df_leads_trafego = carregar_aba(id_dados_leads_ssp, "Página1")
df_pesquisa_captacao = carregar_aba(id_pesquisa_ssp, "Página1")
df_compradores = carregar_aba(id_compradores_ssp, "Compradores")

In [2]:
df_pesquisa_captacao = df_pesquisa_captacao.drop(["Funil"], axis=1, errors='ignore')
df_pesquisa_captacao.rename(columns={'E-mail': 'email'}, inplace=True)
df_pesquisa_captacao.columns = df_pesquisa_captacao.columns.str.lower()

df_pesquisa_captacao['data'] = pd.to_datetime(df_pesquisa_captacao['data'], errors='coerce')
df_pesquisa_captacao['data'] = df_pesquisa_captacao['data'].dt.tz_localize(None)
df_pesquisa_captacao['data'] = df_pesquisa_captacao['data'].dt.normalize()

df_pesquisa_captacao["email"] = df_pesquisa_captacao["email"].str.lower().str.strip()
df_pesquisa_captacao = df_pesquisa_captacao.drop_duplicates(subset='email', keep='first')

df_pesquisa_captacao.info()

<class 'pandas.core.frame.DataFrame'>
Index: 54586 entries, 0 to 56559
Data columns (total 9 columns):
 #   Column                 Non-Null Count  Dtype         
---  ------                 --------------  -----         
 0   data                   54586 non-null  datetime64[ns]
 1   email                  54586 non-null  object        
 2   idade                  54586 non-null  object        
 3   nível                  54586 non-null  object        
 4   situação profissional  54586 non-null  object        
 5   renda                  54586 non-null  object        
 6   escolaridade           54586 non-null  object        
 7   duvida                 54586 non-null  object        
 8   dificuldade            54586 non-null  object        
dtypes: datetime64[ns](1), object(8)
memory usage: 4.2+ MB


In [3]:
df_pesquisa_captacao["renda"] = (
    df_pesquisa_captacao["renda"]
    .replace({
        "O a R$1.000\u2028": "De 0 a 1.000",
        "R$1.000 a R$3.000": "De 1.000 a 3.000",
        "R$3.000 a R$5.000\u2028": "De 3.000 a 5.000",
        "R$5.000 a R$10.000\u2028": "De 5.000 a 10.000",
        "Mais de R$10.000": "Mais de 10.000"
    })
)

df_pesquisa_captacao["escolaridade"] = (
    df_pesquisa_captacao["escolaridade"]
    .replace({
        "Ensino superior incompleto\u2028": "Ensino superior incompleto"
    })
)

In [4]:
# Filtrar o DataFrame para excluir linhas onde o email contém "teste" ou "x@x"
df_pesquisa_captacao = df_pesquisa_captacao[
    ~df_pesquisa_captacao['email'].str.contains('teste|x@x', case=False, na=False)
]

df_pesquisa_captacao.drop(columns=['email']).head()

,data,idade,nível,situação profissional,renda,escolaridade,duvida,dificuldade
2,2025-04-07,33-37,Técnica,Autônomo,De 1.000 a 3.000,Ensino médio completo,O que preciso pra ser aprovado,Tempo
3,2025-04-07,23-27,Técnica,Desempregado(a),Estou desempregado(a),Ensino superior incompleto,.,Separa material
4,2025-04-07,18-22,Superior,Desempregado(a),Estou desempregado(a),Ensino médio completo,O quão difícil é conseguir passar,Nenhuma
5,2025-04-07,28-32,Técnica,Desempregado(a),Estou desempregado(a),Nível técnico completo,Em qual lugar do Brasil pode fazer o concurso?,Tempo
6,2025-04-07,18-22,Técnica,Autônomo,Estou desempregado(a),Ensino médio completo,Se precisa de ensino superior,Nenhuma


In [5]:
df_leads_trafego.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 84054 entries, 0 to 84053
Data columns (total 8 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   Data          84054 non-null  object
 1   Nome          84054 non-null  object
 2   E-mail        84054 non-null  object
 3   UTM Source    84054 non-null  object
 4   UTM Campaign  84054 non-null  object
 5   UTM Medium    84054 non-null  object
 6   UTM Content   84054 non-null  object
 7   Tag           84054 non-null  object
dtypes: object(8)
memory usage: 5.1+ MB


In [6]:
df_leads_trafego.columns = df_leads_trafego.columns.str.strip().str.lower().str.replace(" ", "_")

# Definir colunas UTM
colunas_utms = ['utm_source', 'utm_campaign', 'utm_medium', 'utm_content']

# Função para verificar se uma célula é "vazia de verdade"
def is_vazio(valor):
    if pd.isna(valor):
        return True
    valor = str(valor).strip()
    return (valor == "") or (valor.startswith("{{") and valor.endswith("}}"))

# Função para verificar se TODAS UTMs estão vazias para uma linha
def todas_utms_vazias(row):
    return all(is_vazio(row[col]) for col in colunas_utms)

# Aplicar filtro: manter linhas onde pelo menos uma UTM está preenchida de verdade
df_leads_trafego = df_leads_trafego[
    ~df_leads_trafego.apply(todas_utms_vazias, axis=1)
].copy()

# Remover linhas que contenham 'teste' em qualquer coluna
df_leads_trafego = df_leads_trafego[
    ~df_leads_trafego.apply(lambda row: row.astype(str).str.lower().str.contains('teste').any(), axis=1)
]

# Resetar índice
df_leads_trafego = df_leads_trafego.reset_index(drop=True)

df_leads_trafego.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 59020 entries, 0 to 59019
Data columns (total 8 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   data          59020 non-null  object
 1   nome          59020 non-null  object
 2   e-mail        59020 non-null  object
 3   utm_source    59020 non-null  object
 4   utm_campaign  59020 non-null  object
 5   utm_medium    59020 non-null  object
 6   utm_content   59020 non-null  object
 7   tag           59020 non-null  object
dtypes: object(8)
memory usage: 3.6+ MB


In [7]:
df_leads_trafego.rename(columns={'e-mail': 'email'}, inplace=True)
df_leads_trafego["email"] = df_leads_trafego["email"].str.lower().str.strip()

df_leads_trafego['data'] = pd.to_datetime(df_leads_trafego['data'], errors='coerce')
df_leads_trafego['data'] = df_leads_trafego['data'].dt.tz_localize(None)
df_leads_trafego['data'] = df_leads_trafego['data'].dt.normalize()

df_leads_trafego.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 59020 entries, 0 to 59019
Data columns (total 8 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   data          59020 non-null  datetime64[ns]
 1   nome          59020 non-null  object        
 2   email         59020 non-null  object        
 3   utm_source    59020 non-null  object        
 4   utm_campaign  59020 non-null  object        
 5   utm_medium    59020 non-null  object        
 6   utm_content   59020 non-null  object        
 7   tag           59020 non-null  object        
dtypes: datetime64[ns](1), object(7)
memory usage: 3.6+ MB


C:\Users\Camilo_Bica\AppData\Local\Temp\ipykernel_20604\1702501422.py:4: UserWarning: Parsing dates in %d/%m/%Y format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  df_leads_trafego['data'] = pd.to_datetime(df_leads_trafego['data'], errors='coerce')


In [8]:
# Criar coluna de lançamentos baseada na data
df_leads_trafego['lancamentos'] = df_leads_trafego['data'].apply(
    lambda x: 'SSP-L13' if x.date() >= pd.to_datetime('2025-04-30').date() else 'SSP-L12'
)

# Remover e-mails duplicados dentro de cada lançamento
df_leads_trafego = (
    df_leads_trafego
    .sort_values('data')  # garante ordem cronológica
    .drop_duplicates(subset=['email', 'lancamentos'], keep='first')
)

df_leads_trafego.info()

<class 'pandas.core.frame.DataFrame'>
Index: 58915 entries, 0 to 59019
Data columns (total 9 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   data          58915 non-null  datetime64[ns]
 1   nome          58915 non-null  object        
 2   email         58915 non-null  object        
 3   utm_source    58915 non-null  object        
 4   utm_campaign  58915 non-null  object        
 5   utm_medium    58915 non-null  object        
 6   utm_content   58915 non-null  object        
 7   tag           58915 non-null  object        
 8   lancamentos   58915 non-null  object        
dtypes: datetime64[ns](1), object(8)
memory usage: 4.5+ MB


In [9]:
df_leads_trafego["lancamentos"].value_counts()

lancamentos
SSP-L13    29472
SSP-L12    29443
Name: count, dtype: int64

In [10]:
df_leads_trafego.drop(columns=['email']).head()

,data,nome,utm_source,utm_campaign,utm_medium,utm_content,tag,lancamentos
0,2025-03-31,Hermes,Facebook-Ads,SSP-L12 | CAPTACAO | CBO | FRIO | ALL | BR | 3...,00 | AUTO | LAL | 1% | ENVOLVIMENTO IG 7D,ADS_002_ANUNCIO PETRO 02,SSP-L12,SSP-L12
300,2025-03-31,Johnny,Facebook-Ads,SSP-L12 | CAPTACAO | CBO | FRIO | ALL | BR | 3...,00 | AUTO | LAL | 1% | ENVOLVIMENTO IG 7D,ADS_001_ANUNCIO PETRO 01,SSP-L12,SSP-L12
299,2025-03-31,Willian,Facebook-Ads,SSP-L12 | CAPTACAO | CBO | FRIO | ALL | BR | 3...,00 | AUTO | LAL | 1% | ENVOLVIMENTO IG 14D,ADS_001_ANUNCIO PETRO 01,SSP-L12,SSP-L12
297,2025-03-31,Robson,Facebook-Ads,SSP-L12 | CAPTACAO | CBO | FRIO | ALL | BR | 3...,00 | AUTO | LAL | 1% | ENVOLVIMENTO IG 14D,ADS_001_ANUNCIO PETRO 01,SSP-L12,SSP-L12
296,2025-03-31,Zidane,Facebook-Ads,SSP-L12 | CAPTACAO | CBO | FRIO | ALL | BR | 3...,00 | AUTO | LAL | 1% | ENVOLVIMENTO IG 14D,ADS_001_ANUNCIO PETRO 01,SSP-L12,SSP-L12


In [11]:
df_compradores.rename(columns={'email contato': 'email'}, inplace=True)
df_compradores["email"] = df_compradores["email"].str.lower().str.strip()
df_compradores = df_compradores.drop_duplicates(subset='email', keep='first')

df_compradores.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 593 entries, 0 to 592
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   email   593 non-null    object
dtypes: object(1)
memory usage: 4.8+ KB


In [12]:
df_pesquisa_compradores_l12 = df_pesquisa_captacao[
    df_pesquisa_captacao["email"].isin(df_compradores["email"])
].copy()

In [13]:
df_pesquisa_compradores_l12.info()

<class 'pandas.core.frame.DataFrame'>
Index: 199 entries, 12 to 48799
Data columns (total 9 columns):
 #   Column                 Non-Null Count  Dtype         
---  ------                 --------------  -----         
 0   data                   199 non-null    datetime64[ns]
 1   email                  199 non-null    object        
 2   idade                  199 non-null    object        
 3   nível                  199 non-null    object        
 4   situação profissional  199 non-null    object        
 5   renda                  199 non-null    object        
 6   escolaridade           199 non-null    object        
 7   duvida                 199 non-null    object        
 8   dificuldade            199 non-null    object        
dtypes: datetime64[ns](1), object(8)
memory usage: 15.5+ KB


In [14]:
df_pesquisa_compradores_l12.drop(columns=['email']).head()

,data,idade,nível,situação profissional,renda,escolaridade,duvida,dificuldade
12,2025-04-07,18-22,Superior,Funcionário de empresa privada,De 1.000 a 3.000,Ensino superior completo,A quais vagas posso concorrer tendo tecnólogo ...,Tempo
60,2025-04-07,38-45,Técnica,Funcionário de empresa privada,De 1.000 a 3.000,Ensino médio completo,Tenho o curso técnico em contabilidade servi p...,Memorização
270,2025-04-07,28-32,Técnica,Funcionário de empresa privada,De 1.000 a 3.000,Ensino médio completo,Em qual cidade irei fazer o concurso?,Não sei o que estudar
295,2025-04-07,28-32,Técnica,Desempregado(a),De 1.000 a 3.000,Ensino superior completo,Quando vai sair o edital,Foco
309,2025-04-07,46-52,Técnica,Funcionário de empresa privada,De 3.000 a 5.000,Nível técnico completo,A prova,Tempo


In [15]:
df_compradores_nao_respondentes = df_compradores[
    ~df_compradores["email"].isin(df_pesquisa_captacao["email"])
].copy()

In [16]:
df_compradores_nao_respondentes.info()

<class 'pandas.core.frame.DataFrame'>
Index: 394 entries, 0 to 591
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   email   394 non-null    object
dtypes: object(1)
memory usage: 6.2+ KB


In [17]:
# Merge mantendo os lançamentos da pesquisa
df_pesquisa_captacao = df_pesquisa_captacao.merge(
    df_leads_trafego[['email'] + colunas_utms + ['lancamentos']],
    on='email',
    how='left'
)

df_pesquisa_compradores_l12 = df_pesquisa_compradores_l12.merge(
    df_leads_trafego[["email"] + colunas_utms + ["lancamentos"]],
    on="email",
    how="left"
)

In [18]:
emails_pesquisa = set(df_pesquisa_captacao['email'].unique())
emails_trafego = set(df_leads_trafego['email'].unique())
emails_aluno = set(df_pesquisa_compradores_l12['email'].unique())

# Quantos e-mails da pesquisa não estão no tráfego?
len(emails_pesquisa - emails_trafego)

20749

In [19]:
df_pesquisa_captacao.drop(columns=['email']).head()

,data,idade,nível,situação profissional,renda,escolaridade,duvida,dificuldade,utm_source,utm_campaign,utm_medium,utm_content,lancamentos
0,2025-04-07,33-37,Técnica,Autônomo,De 1.000 a 3.000,Ensino médio completo,O que preciso pra ser aprovado,Tempo,NaN,NaN,NaN,NaN,NaN
1,2025-04-07,23-27,Técnica,Desempregado(a),Estou desempregado(a),Ensino superior incompleto,.,Separa material,Facebook-Ads,SSP-L12 | CAPTACAO | CBO | FRIO | ALL | BR | 3...,00 | AUTO | LAL | 1% | ENVOLVIMENTO IG 7D,ADS_001_ANUNCIO PETRO 01,SSP-L12
2,2025-04-07,18-22,Superior,Desempregado(a),Estou desempregado(a),Ensino médio completo,O quão difícil é conseguir passar,Nenhuma,Facebook-Ads,SSP-L12 | CAPTACAO | CBO | QUENTE | ALL | BR |...,00 | AUTO | ENVOLVIMENTO | 90D,ADS_006_ANUNCIO PETRO 06,SSP-L12
3,2025-04-07,28-32,Técnica,Desempregado(a),Estou desempregado(a),Nível técnico completo,Em qual lugar do Brasil pode fazer o concurso?,Tempo,Facebook-Ads,SSP-L12 | CAPTACAO | CBO | QUENTE | ALL | BR |...,00 | AUTO | ENVOLVIMENTO | 30D,ADS_001_ANUNCIO PETRO 01,SSP-L12
4,2025-04-07,18-22,Técnica,Autônomo,Estou desempregado(a),Ensino médio completo,Se precisa de ensino superior,Nenhuma,NaN,NaN,NaN,NaN,NaN


In [20]:
df_pesquisa_compradores_l12.drop(columns=['email']).head()

,data,idade,nível,situação profissional,renda,escolaridade,duvida,dificuldade,utm_source,utm_campaign,utm_medium,utm_content,lancamentos
0,2025-04-07,18-22,Superior,Funcionário de empresa privada,De 1.000 a 3.000,Ensino superior completo,A quais vagas posso concorrer tendo tecnólogo ...,Tempo,Youtube,,Descricao,,SSP-L12
1,2025-04-07,38-45,Técnica,Funcionário de empresa privada,De 1.000 a 3.000,Ensino médio completo,Tenho o curso técnico em contabilidade servi p...,Memorização,NaN,NaN,NaN,NaN,NaN
2,2025-04-07,28-32,Técnica,Funcionário de empresa privada,De 1.000 a 3.000,Ensino médio completo,Em qual cidade irei fazer o concurso?,Não sei o que estudar,Youtube,,Descricao,,SSP-L12
3,2025-04-07,28-32,Técnica,Desempregado(a),De 1.000 a 3.000,Ensino superior completo,Quando vai sair o edital,Foco,Facebook-Ads,SSP-L12 | CAPTACAO | CBO | QUENTE | ALL | BR |...,00 | AUTO | ENVOLVIMENTO | 90D,ADS_007_ANUNCIO PETRO 07,SSP-L12
4,2025-04-07,46-52,Técnica,Funcionário de empresa privada,De 3.000 a 5.000,Nível técnico completo,A prova,Tempo,Facebook-Ads,SSP-L12 | CAPTACAO | CBO | FRIO | ALL | BR | 3...,00 | AUTO | LAL | 1% | ENVOLVIMENTO IG 14D,ADS_001_ANUNCIO PETRO 01,SSP-L12


In [21]:
df_pesquisa_captacao.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 55092 entries, 0 to 55091
Data columns (total 14 columns):
 #   Column                 Non-Null Count  Dtype         
---  ------                 --------------  -----         
 0   data                   55092 non-null  datetime64[ns]
 1   email                  55092 non-null  object        
 2   idade                  55092 non-null  object        
 3   nível                  55092 non-null  object        
 4   situação profissional  55092 non-null  object        
 5   renda                  55092 non-null  object        
 6   escolaridade           55092 non-null  object        
 7   duvida                 55092 non-null  object        
 8   dificuldade            55092 non-null  object        
 9   utm_source             34343 non-null  object        
 10  utm_campaign           34343 non-null  object        
 11  utm_medium             34343 non-null  object        
 12  utm_content            34343 non-null  object        
 13  l

In [22]:
# Contar quantas linhas têm todas as UTMs como NaN
qtd_todas_utms_nan = df_pesquisa_compradores_l12[colunas_utms].isna().all(axis=1).sum()

print(f"Quantidade de linhas com TODAS as UTMs NaN: {qtd_todas_utms_nan}")

Quantidade de linhas com TODAS as UTMs NaN: 57


In [23]:
"""
model = SentenceTransformer("all-MiniLM-L6-v2")

# Mapas distintos por coluna
mapeamentos = {
    'problema_aprender': mapeamento_problema_aprender,
    'profissao': mapeamento_profissoes,
    'fala_outro_idioma': mapeamento_outros_idiomas,
    'motivo_fluencia_espanhol': mapeamento_motivo_fluencia,
    'escolaridade': mapeamento_escolaridade
}

for coluna, mapeamento in mapeamentos.items():
    nome_coluna_categoria = f"{coluna}_categoria"

    # Expandir mapeamento para esta coluna
    mapeamento_expandido, mapeamento_embeddings = preparar_para_categoria(mapeamento, model)

    # Categorização com o mapeamento certo
    df_pesquisa_captacao_lancamentos.loc[:, nome_coluna_categoria] = categorizar_coluna_batch(
        df_pesquisa_captacao_lancamentos[coluna],
        mapeamento_expandido,
        mapeamento_embeddings,
        model,
        threshold=0.6,
        desc=coluna
    )
"""

'\nmodel = SentenceTransformer("all-MiniLM-L6-v2")\n\n# Mapas distintos por coluna\nmapeamentos = {\n    \'problema_aprender\': mapeamento_problema_aprender,\n    \'profissao\': mapeamento_profissoes,\n    \'fala_outro_idioma\': mapeamento_outros_idiomas,\n    \'motivo_fluencia_espanhol\': mapeamento_motivo_fluencia,\n    \'escolaridade\': mapeamento_escolaridade\n}\n\nfor coluna, mapeamento in mapeamentos.items():\n    nome_coluna_categoria = f"{coluna}_categoria"\n\n    # Expandir mapeamento para esta coluna\n    mapeamento_expandido, mapeamento_embeddings = preparar_para_categoria(mapeamento, model)\n\n    # Categorização com o mapeamento certo\n    df_pesquisa_captacao_lancamentos.loc[:, nome_coluna_categoria] = categorizar_coluna_batch(\n        df_pesquisa_captacao_lancamentos[coluna],\n        mapeamento_expandido,\n        mapeamento_embeddings,\n        model,\n        threshold=0.6,\n        desc=coluna\n    )\n'

In [24]:
"""# Exibir strings completas
pd.set_option("display.max_colwidth", None)

# Colunas categorizadas e respectivas colunas originais
categorias = {
    "problema_aprender": "problema_aprender_categoria",
    "profissao": "profissao_categoria",
    "fala_outro_idioma": "fala_outro_idioma_categoria",
    "motivo_fluencia_espanhol": "motivo_fluencia_espanhol_categoria",
    "escolaridade": "escolaridade_categoria"
}

# Categorias problemáticas
categorias_problema = ["Outros"]

# Loop para filtrar e exibir como DataFrame formatado
for original_col, categoria_col in categorias.items():
    for problema in categorias_problema:
        print(f"\n=== Valores de '{original_col}' categorizados como '{problema}' ===")

        filtro = df_pesquisa_captacao_lancamentos[categoria_col] == problema
        valores = df_pesquisa_captacao_lancamentos.loc[filtro, original_col].value_counts()

        # Envolver cada valor com aspas
        valores.index = [f'"{val}"' for val in valores.index]

        # Converter para DataFrame para visualização completa
        valores_df = valores.reset_index().head(300)
        valores_df.columns = ['valor_original', 'frequencia']

        # Exibir o resultado completo
        print(valores_df.to_string(index=False))
"""

'# Exibir strings completas\npd.set_option("display.max_colwidth", None)\n\n# Colunas categorizadas e respectivas colunas originais\ncategorias = {\n    "problema_aprender": "problema_aprender_categoria",\n    "profissao": "profissao_categoria",\n    "fala_outro_idioma": "fala_outro_idioma_categoria",\n    "motivo_fluencia_espanhol": "motivo_fluencia_espanhol_categoria",\n    "escolaridade": "escolaridade_categoria"\n}\n\n# Categorias problemáticas\ncategorias_problema = ["Outros"]\n\n# Loop para filtrar e exibir como DataFrame formatado\nfor original_col, categoria_col in categorias.items():\n    for problema in categorias_problema:\n        print(f"\n=== Valores de \'{original_col}\' categorizados como \'{problema}\' ===")\n\n        filtro = df_pesquisa_captacao_lancamentos[categoria_col] == problema\n        valores = df_pesquisa_captacao_lancamentos.loc[filtro, original_col].value_counts()\n\n        # Envolver cada valor com aspas\n        valores.index = [f\'"{val}"\' for val in

In [25]:
"""df_pesquisa_captacao_lancamentos = df_pesquisa_captacao_lancamentos.drop([
    'profissao',
    'fala_outro_idioma',
    'motivo_fluencia_espanhol',
    'escolaridade',
    'problema_aprender'
], axis=1, errors='ignore')
"""

"df_pesquisa_captacao_lancamentos = df_pesquisa_captacao_lancamentos.drop([\n    'profissao',\n    'fala_outro_idioma',\n    'motivo_fluencia_espanhol',\n    'escolaridade',\n    'problema_aprender'\n], axis=1, errors='ignore')\n"

In [26]:
from gspread_dataframe import set_with_dataframe
from datetime import datetime

# Carrega as variáveis de ambiente
load_dotenv()

# Pega o caminho de forma segura
credenciais_path = os.getenv("GOOGLE_CREDENTIALS_PATH")

# === Escopos de acesso ===
scopes = [
    "https://www.googleapis.com/auth/spreadsheets",
    "https://www.googleapis.com/auth/drive"
]

# === Autenticar ===
creds = service_account.Credentials.from_service_account_file(
    credenciais_path,
    scopes=scopes
)
client = gspread.authorize(creds)

# === Função para criar nova planilha e carregar dados ===
def criar_planilha_e_enviar(df, nome_base):
    nome_final = nome_base
    
    # Cria nova planilha
    planilha = client.create(nome_final)
    
    # Compartilha com seu e-mail pessoal (aqui você coloca o seu)
    planilha.share('camilobf2@gmail.com', perm_type='user', role='writer')  # <<< Trocar pelo seu email do Gmail

    # Preenche a primeira aba
    aba = planilha.sheet1
    aba.update_title("Dados")
    set_with_dataframe(aba, df)
    
    print(f"✅ Nova planilha criada: {nome_final}")
    print(f"🔗 Link: https://docs.google.com/spreadsheets/d/{planilha.id}/edit")

# === Geração dos arquivos ===
#criar_planilha_e_enviar(df_pesquisa_captacao, "pesquisa_captacao")
#criar_planilha_e_enviar(df_pesquisa_compradores_l12, "pesquisa_aluno")
#criar_planilha_e_enviar(df_leads_invest_trafego, "invest_trafego")

In [27]:
# === Função para atualizar uma planilha existente ===
def atualizar_planilha_existente(df: pd.DataFrame, sheet_id: str, aba_nome: str = "Dados"):
    """
    Atualiza uma aba específica de uma planilha no Google Sheets.
    
    - df: DataFrame com os dados que serão enviados
    - sheet_id: ID da planilha (o que vem na URL depois de "/d/")
    - aba_nome: Nome da aba que será atualizada (default: 'Dados')
    """
    try:
        planilha = client.open_by_key(sheet_id)
        try:
            aba = planilha.worksheet(aba_nome)
        except gspread.WorksheetNotFound:
            aba = planilha.add_worksheet(title=aba_nome, rows="1000", cols="20")
        
        aba.clear()  # Limpa dados antigos
        set_with_dataframe(aba, df)
        
        print(f"✅ Planilha atualizada: https://docs.google.com/spreadsheets/d/{sheet_id}/edit")
    except Exception as e:
        print(f"❌ Erro ao atualizar a planilha: {e}")

# === IDs das planilhas (FIXOS, preencha certinho aqui) ===
id_pesquisa_captacao = "1ukLwu8SoP0U3uirB6w1Ca3TPEITY50c558xNUUN3kj4"
id_pesquisa_aluno = "1GDCAa1fiflDIBnRY9rrdY9ghKwS6DPzxIG9FYGZEsQU"


# === Atualizar todas as planilhas ===
atualizar_planilha_existente(df_pesquisa_captacao, id_pesquisa_captacao)
atualizar_planilha_existente(df_pesquisa_compradores_l12, id_pesquisa_aluno)

✅ Planilha atualizada: https://docs.google.com/spreadsheets/d/1ukLwu8SoP0U3uirB6w1Ca3TPEITY50c558xNUUN3kj4/edit


✅ Planilha atualizada: https://docs.google.com/spreadsheets/d/1GDCAa1fiflDIBnRY9rrdY9ghKwS6DPzxIG9FYGZEsQU/edit


In [28]:
from datetime import datetime
import pytz

fuso_brasil = pytz.timezone("America/Sao_Paulo")
agora_brasil = datetime.now(fuso_brasil)

with open("ultima_atualizacao.txt", "w") as f:
    f.write(agora_brasil.strftime("%Y-%m-%d %H:%M:%S"))

In [29]:
try:
    with open("ultima_atualizacao.txt", "r") as f:
        texto = f.read()
        data_atualizacao = datetime.strptime(texto, "%Y-%m-%d %H:%M:%S")
        data_atualizacao_formatada = data_atualizacao.strftime("%d/%m/%Y %H:%M")
except Exception:
    data_atualizacao_formatada = "Desconhecida"

In [30]:
f"**Última atualização:** {data_atualizacao_formatada}"

'**Última atualização:** 12/05/2025 08:55'

In [31]:
colunas_excluir = ["data", "email"]

for coluna in df_pesquisa_captacao.columns:
    if coluna not in colunas_excluir:
        print(f"\nColuna: {coluna}")
        print(df_pesquisa_captacao[coluna].value_counts(dropna=False))


Coluna: idade
idade
23-27         11660
18-22         10919
28-32          9638
38-45          9082
33-37          7961
46-52          4169
53 ou mais     1663
Name: count, dtype: int64

Coluna: nível
nível
Técnica     41194
Superior    13898
Name: count, dtype: int64

Coluna: situação profissional
situação profissional
Funcionário de empresa privada                          23283
Autônomo                                                13746
Desempregado(a)                                         12189
Funcionário público                                      3859
Empresário                                               1152
Já sou funcionário terceirizado do sistema Petrobras      863
Name: count, dtype: int64

Coluna: renda
renda
De 1.000 a 3.000         27824
De 3.000 a 5.000         10535
De 0 a 1.000              6827
Estou desempregado(a)     6063
De 5.000 a 10.000         3279
Mais de 10.000             564
Name: count, dtype: int64

Coluna: escolaridade
escolaridade
Ensino médi

In [32]:
colunas_excluir = ["data", "email"]

for coluna in df_pesquisa_compradores_l12.columns:
    if coluna not in colunas_excluir:
        print(f"\nColuna: {coluna}")
        print(df_pesquisa_compradores_l12[coluna].value_counts(dropna=False))


Coluna: idade
idade
23-27         51
38-45         40
28-32         35
33-37         32
46-52         24
18-22         17
53 ou mais     6
Name: count, dtype: int64

Coluna: nível
nível
Técnica     173
Superior     32
Name: count, dtype: int64

Coluna: situação profissional
situação profissional
Funcionário de empresa privada                          106
Desempregado(a)                                          41
Autônomo                                                 33
Empresário                                               12
Funcionário público                                      11
Já sou funcionário terceirizado do sistema Petrobras      2
Name: count, dtype: int64

Coluna: renda
renda
De 1.000 a 3.000         82
De 3.000 a 5.000         60
De 5.000 a 10.000        27
Estou desempregado(a)    22
De 0 a 1.000             11
Mais de 10.000            3
Name: count, dtype: int64

Coluna: escolaridade
escolaridade
Ensino médio completo         75
Ensino superior completo      62
